# Lab-5-1 : RNN Practice-1

In [1]:
# 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성하는 프로그램입니다.
import tensorflow as tf
import numpy as np


char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# making a encoding and decoding array
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# training data 
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [7]:
print(num_dic)

{'i': 8, 'b': 1, 't': 19, 'g': 6, 'c': 2, 'h': 7, 'v': 21, 'o': 14, 'r': 17, 's': 18, 'q': 16, 'y': 24, 'f': 5, 'k': 10, 'l': 11, 'u': 20, 'm': 12, 'e': 4, 'a': 0, 'x': 23, 'w': 22, 'd': 3, 'p': 15, 'n': 13, 'j': 9, 'z': 25}


In [8]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        
        # making the input and target batch
        # wor -->  [22, 14, 17]   woo --> [22, 14, 14]   dee --->  [3, 4, 4]   div ---> [3, 8, 21] ...
        input = [num_dic[n] for n in seq[:-1]]
        
        # d --> 3, d --> 3, p ---> 15,  e---> 4, d ---> 3 ...
        target = num_dic[seq[-1]]
        
        
        # make a one-hot encoding
        # if input is [0, 1, 2]:
        # [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
        #  [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]
        input_batch.append(np.eye(dic_len)[input])
        
        
        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
        # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
        # 이 예제에서 사용할 손실 함수인 sparse_softmax_cross_entropy_with_logits 는
        # one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨주면 됩니다.
        target_batch.append(target)

    return input_batch, target_batch

In [9]:
#########
# Setting the Option
######
learning_rate = 0.01
n_hidden = 128
total_epoch = 30

# Number sequence in RNN=> 3
n_step = 3

# size of input : 26 (because it is the one-encoding of the alphabets
# ex) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
# The output is also of size 26
n_input = n_class = dic_len

In [3]:
#########
# Make the Network 
######
X = tf.placeholder(tf.float32, [None, n_step, n_input])
# Because we use the sparse_softmax_cross_entropy_with_logits method
# the input for Y is not of one-hot vector form 
# but just a one-dimensional vector
# [3] [3] [15] [4] ...
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# Make the RNN cell.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# Use Dropout to prevent overfitting
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)

# Make another cell
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# Make an RNN cell with cell1 and cell2
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# Make an RNN using tf.nn.dynamic_rnn function.
# time_major=True
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# change the first and second dimension of outputs  ([n_step, batch_size, n_hidden]->[batch_size, n_step, n_hidden])
outputs = tf.transpose(outputs, [1, 0, 2])

# take only the hidden state of the last step [batch_size, n_step, n_hidden] --> [batch_size, n_step] (for the last step)
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [ ]:
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#########
# Training the network
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('Optimization finish!')

In [ ]:
#########
# Result 
######
# Label is integer, so the predicted value is also casted as integer
prediction = tf.cast(tf.argmax(model, 1), tf.int32)

# Compare the input.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== Predict Result ===')
print('input:', [w[:3] + ' ' for w in seq_data])
print('predicted:', predict_words)
print('accuracy:', accuracy_val)